In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import warnings
from datetime import datetime
import country_converter as coco
warnings.simplefilter('ignore')
from sklearn.linear_model import LinearRegression,Ridge,Lasso,LogisticRegression
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import pickle

In [2]:
df = pd.read_csv("cleaned_df.csv")
df = df.drop(columns=['Unnamed: 0'])

df = df.rename(columns = {
        'area_in_h': 'Area',
        'production_in_t':'Production',
        'yield_in_kg_per_h':'Yield'
    }
)

df

,Country,Item,Year,Production,Area,Yield,Category,continent
0,Afghanistan,"Almonds, in shell",2020,39307.0,22134.0,1776.0,Nuts,Asia
1,Afghanistan,"Anise, badian, coriander, cumin, caraway, fenn...",2019,17808.0,25293.0,704.0,Spices,Asia
2,Afghanistan,"Anise, badian, coriander, cumin, caraway, fenn...",2020,17966.0,25558.0,703.0,Spices,Asia
3,Afghanistan,"Anise, badian, coriander, cumin, caraway, fenn...",2021,17876.0,25357.0,705.0,Spices,Asia
4,Afghanistan,"Anise, badian, coriander, cumin, caraway, fenn...",2022,17883.0,25403.0,704.0,Spices,Asia
...,...,...,...,...,...,...,...,...
31001,Zimbabwe,Watermelons,2019,92975.0,3719.0,25000.0,Fruits,Africa
31002,Zimbabwe,Watermelons,2021,139000.0,4430.0,31377.0,Fruits,Africa
31003,Zimbabwe,Watermelons,2022,162100.0,4790.0,33841.0,Fruits,Africa
31004,Zimbabwe,Watermelons,2023,150600.0,4610.0,32668.0,Fruits,Africa


In [6]:
# Encoding

le_country = LabelEncoder()
le_item = LabelEncoder()

df['country_encoded'] = le_country.fit_transform(df['Country'])
df['Item_encoded'] = le_item.fit_transform(df['Item'])

In [7]:
from sklearn.ensemble import RandomForestRegressor

x = df[['country_encoded','Item_encoded','Year','Area','Yield']]
y = df['Production']

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

rf = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=-1)
rf.fit(x_train,y_train)
y_rf_predict = rf.predict(x_test)

rf_score = rf.score(x_test,y_test)
mae = mean_absolute_error(y_test,y_rf_predict)
mse = mean_squared_error(y_test,y_rf_predict)
rs = r2_score(y_test,y_rf_predict)

print(f'the rf score is {rf_score}')
print(f'mae:{mae}, mse:{mse}, r2_score:{rs}')

the rf score is 0.9986740422921735
mae:506.4028641874357, mse:6387587.867959847, r2_score:0.9986740422921735


In [ ]:
import pickle

with open('regression_model.pkl','wb') as f:
    pickle.dump(rf,f)

with open("regression_model.pkl", "rb") as file:
    loaded_model = pickle.load(file)

In [13]:
with open("crop_label_encoder.pkl", "wb") as f:
    pickle.dump({'country': le_country, 'item': le_item}, f)
    
with open("crop_label_encoder.pkl", "rb") as f:
    encoders = pickle.load(f)
